In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from pathlib import Path

def load_and_merge_data(excel_path: Path) -> pd.DataFrame:
    """
    Loads 'Visite' and 'Anagrafica' sheets from an Excel file and merges them.

    Args:
        excel_path: Path to the Excel file.

    Returns:
        A merged pandas DataFrame.
    """
    xls = pd.ExcelFile(excel_path)
    sheet_4 = pd.read_excel(xls, 'Visite')
    sheet_1 = pd.read_excel(xls, 'Anagrafica')

    # Merge the sheets on 'Paziente ID' to combine clinical and demographic data
    final_data = pd.merge(sheet_4, sheet_1, how='left', on=['Paziente ID'])
    return final_data

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Selects important features, translates headers, handles missing values,
    converts data types, and encodes categorical features.

    Args:
        df: The input DataFrame.

    Returns:
        A preprocessed pandas DataFrame.
    """
    # Select important features
    df = df[[
        "Id", "Paziente ID", 'Data visita', 'Piramidale', 'Cerebellare',
        'Troncoencefalica', 'Sensitiva', 'Sfinteriche', 'Visiva', 'Mentali',
        'Deambulazione', 'Punteggio EDSS valutato dal clinico', 'Data di nascita',
        'Data di morte', 'Sesso', 'SM in età pedriatrica'
    ]].copy()

    # Translate headers for clarity and consistency
    df.columns = [
        'Id', 'Patient_ID', 'Date_of_visit', 'Pyramidal', 'Cerebellar',
        'Thronchioencephalic', 'Sensitive', 'Sphincteric', 'Visual', 'Mental',
        'Deambulation', 'EDSS_score_assessed_by_clinician', 'Date_of_birth',
        'Date_of_death', 'Sex', 'MS_in_pediatric_age'
    ]

    # Remove rows where the target value (EDSS_score_assessed_by_clinician) is missing
    df = df[df['EDSS_score_assessed_by_clinician'].notna()]

    # Convert relevant columns to datetime objects
    df['Date_of_visit'] = pd.to_datetime(df['Date_of_visit'])
    df['Date_of_birth'] = pd.to_datetime(df['Date_of_birth'])

    # Use LabelEncoder to convert 'Sex' and 'MS_in_pediatric_age' to numerical labels
    le = preprocessing.LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])  # 0 = Female, 1 = Male
    df['MS_in_pediatric_age'] = le.fit_transform(df['MS_in_pediatric_age']) # 0 = No, 1 = Yes

    return df

def remove_patients_with_high_missingness(df: pd.DataFrame) -> pd.DataFrame:
    """
    Deletes patients with only one observation that is largely missing (>=50%).

    Args:
        df: The input DataFrame.

    Returns:
        A DataFrame with patients having high missingness removed.
    """
    initial_patient_ids = df['Patient_ID'].nunique()
    print(f"Initial number of unique patients: {initial_patient_ids}")

    patients_to_drop = []
    for patient_id in np.unique(df["Patient_ID"]):
        patient_data = df[df["Patient_ID"] == patient_id]
        if len(patient_data) == 1:
            missing_rate = (patient_data.isnull().sum(axis=1) / patient_data.shape[1]) * 100
            if missing_rate.values[0] >= 50:
                patients_to_drop.append(patient_id)

    if patients_to_drop:
        df = df[~df['Patient_ID'].isin(patients_to_drop)].copy()
        print(f"Removed {len(patients_to_drop)} patients with high missingness in single observation.")
    else:
        print("No patients with high missingness in single observation found for removal.")

    final_patient_ids = df['Patient_ID'].nunique()
    print(f"Final number of unique patients after removal: {final_patient_ids}")
    return df

def calculate_age(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the age of the patient at the time of visit.

    Args:
        df: The input DataFrame with 'Date_of_visit' and 'Date_of_birth' columns.

    Returns:
        DataFrame with an 'Age' column added.
    """
    df['Age'] = (df['Date_of_visit'] - df['Date_of_birth']).dt.days / 365.25
    df['Age'] = df['Age'].astype(int)
    return df

def main():
    """
    Main function to orchestrate the data loading, preprocessing, and saving.
    """
    # Define the path to the Excel file
    excel_file_path = Path('C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/Clinical_data/Original.xlsx')

    if not excel_file_path.exists():
        print(f"Error: The file '{excel_file_path}' does not exist. Please check the path.")
        return

    # Load and merge data
    print("Loading and merging data...")
    merged_data = load_and_merge_data(excel_file_path)
    print(f"Initial data shape after merge: {merged_data.shape}")

    # Preprocess data
    print("Preprocessing data...")
    processed_data = preprocess_data(merged_data)
    print(f"Data shape after preprocessing: {processed_data.shape}")

    # Remove patients with high missingness
    print("Checking for and removing patients with high missingness...")
    cleaned_data = remove_patients_with_high_missingness(processed_data)
    print(f"Data shape after removing highly missing patients: {cleaned_data.shape}")

    # Calculate age
    print("Calculating patient age...")
    final_cleaned_data = calculate_age(cleaned_data)

    # Save the processed data to a CSV file
    output_csv_path = Path("Updated_MS.csv")
    final_cleaned_data.to_csv(output_csv_path, index=False)
    print(f"\nCleaned data saved successfully to '{output_csv_path}'")
    print(f"Final data shape: {final_cleaned_data.shape}")
    print("\nFirst 5 rows of the cleaned data:")
    print(final_cleaned_data.head())

if __name__ == "__main__":
    main()